In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2022-01'
post_data['gender'] = 'all'
post_data['page'] = None
actors_info_list = list()

for i in range(1,11):
    post_data['page'] = i
    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.text, 'html.parser')
    actors = soup.select('li.people_li div.name')
    strong_scores = soup.select('strong')
    ranks = soup.select('span.grade')

    movies = [list(map(lambda x : x.text, soup.select('ul.mov_list')[i].select('li > a > span'))) for i in range(len(actors))]

    for actor, score, movie, rank in zip(actors, strong_scores, movies,ranks):
        actor_link = 'http://www.cine21.com' + actor.select_one('a')['href'] # 태그 내 속성 접근
        response_actor = requests.get(actor_link) # 배우 상세 페이지 이동
        soup_actor = BeautifulSoup(response_actor.text, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        actor_details = default_info.select('li')

        actor_info_dict = dict()
        actor_info_dict['이름'] = re.sub('\(\w*\)', '', actor.text)
        actor_info_dict['랭킹'] = rank.text
        actor_info_dict['흥행점수'] = score.text
        actor_info_dict['출현영화'] = movie

        for actor_detail in actor_details:
            actor_detail_key = actor_detail.select_one('span.tit').text # 상세정보 key값
            actor_detail_value = re.sub('<span.*>.*</span>', '', str(actor_detail)) # span 태그 제거
            actor_detail_value = re.sub('<.*?>', '', actor_detail_value) # 상세정보 value값
            actor_info_dict[actor_detail_key] = actor_detail_value

        actors_info_list.append(actor_info_dict)

In [3]:
pd.DataFrame(actors_info_list)

,이름,랭킹,흥행점수,출현영화,다른 이름,직업,생년월일,성별,홈페이지,신장/체중,학교,취미,소속사,특기,원어명
0,강하늘,1,"79,392","[해적: 도깨비 깃발, 해피 뉴 이어]",김하늘,배우,1990-02-21,남,\nhttp://weibo.com/galpos3?is_hot=1\n,"181cm, 70kg",중앙대학교 연극학과,NaN,NaN,NaN,NaN
1,한효주,2,"67,602",[해적: 도깨비 깃발],NaN,배우,1987-02-22,여,\nhttps://www.facebook.com/hhj.official\n,170cm,동국대학교 연극영화,영화감상,NaN,NaN,NaN
2,이광수,3,"59,677","[해적: 도깨비 깃발, 해피 뉴 이어]",NaN,배우,1985-07-14,남,\nhttps://twitter.com/masijacoke85\nhttps://ww...,190cm,NaN,NaN,킹콩엔터테인먼트,NaN,NaN
3,권상우,4,"48,287",[해적: 도깨비 깃발],Kwon Sang Woo,배우,1976-08-05,남,NaN,"183cm, 72kg",한남대학교 미술교육학 학사,"수영, 헬스, 복싱",벨액터스 엔터테인먼트,농구,NaN
4,조진웅,5,"45,244","[경관의 피, 1984 최동원]",조원준,배우,1976-03-03,남,\nhttp://www.facebook.com/saram.chojinwoong\n,"185cm, 98kg",경성대학교 연극영화,NaN,㈜사람엔터테인먼트,진도 북춤,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,최수민,66,867,[해피 뉴 이어],NaN,성우,NaN,여,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,한서울,67,800,[해피 뉴 이어],NaN,배우,NaN,여,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,신순애,68,754,[미싱타는 여자들],NaN,NaN,NaN,여,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,강영주,69,734,[해피 뉴 이어],NaN,배우,1993-12-12,여,NaN,NaN,NaN,NaN,NaN,NaN,NaN
